1 定义工具函数

配置好环境变量后，我们就可以通过Assistant API来构建一个Agent了。
首先我们需要定义一些工具函数，假设我们的答疑机器人有三个功能，查询员工信息，发送请假申请，查询公司规章制度。
我们需要定义三个工具函数。

1.1 查询员工信息

你希望将用户的提问作为工具函数的输入，并生成对应的SQL语句（这一步骤也叫NL2SQL，即自然语言转SQL语句），再使用SQL语句去数据库中查询，将查询结果作为工具函数的输出。为了帮助你更多的关注到Agent的内容，我们模拟一个查询步骤，而不实际去数据库中查询。

假设员工表名为employee，字段包括department（部门）、name（姓名）、HR。

In [ ]:
from llama_index.llms.dashscope import DashScope
from llama_index.core.base.llms.types import MessageRole, ChatMessage

def query_employee_info(query):
  llm = DashScope(model_name="qwen-plus")
  messages = [
    ChatMessage(role=MessageRole.SYSTEM, content='''你有一个表叫employees，记录公司的员工信息，这个表有department（部门）、name（姓名）、HR三个字段。
      你需要根据用户输入生成sql语句进行查询,你一定不能生成sql语句之外的内容，也不要把```sql```这个信息加上。
    '''),
    ChatMessage(role=MessageRole.USER, content=query)
  ]
  SQL_output = llm.chat(messages).message.content.lower()
  print(f'SQL statement is: {SQL_output}')
  # 2. 根据SQL语句去查询数据库（此处为模拟查询），并返回结果
  if SQL_output == "select count(*) from employees where department = '教育部门'":
      return "教育部门共有66名员工。"
  if SQL_output == "SELECT HR FROM employees WHERE name = '张三'".lower():
      return "张三的HR是李四。"
  if SQL_output == "select department from employees where name = '王五'".lower():
      return "王五的部门是后勤部。"
  else:
      return "抱歉，我暂时无法回答您的问题。"

# query_employee_info("王五在哪个部门？")
# print("*"*12)
query_employee_info("教育部门有几个人")

************
SQL statement is: select count(*) from employees where department = '教育部门'


'教育部门共有66名员工。'

1.2 发送请假申请

你希望这个工具函数将员工输入的请假日期作为输入参数，并返回一个申请成功的字符串。我们在此模拟一个申请步骤，而不实际去公司系统中提交请假申请。

In [23]:
def send_leave_application(date):
  '''
  输入请假时间，输出请假申请发送结果
  '''
  return f'已为你发送请假申请，请假日期是{date}'

# 测试一下这个函数
print(send_leave_application("明后两天"))

已为你发送请假申请，请假日期是明后两天


1.3 查询公司规章制度

你希望这个工具函数以用户的提问作为输入，输出是根据之前教程创建的RAG答疑机器人给出的回复。

In [25]:
import sys
sys.path.append("..")
from chatbot.rag import load_index

def query_company_info(query):
  '''
  输入用户提问，输出公司信息查询结果
  '''
  # 使用封装好的函数加载索引
  index = load_index()
  query_engine = index.as_query_engine(
    llm=DashScope(model_name="qwen-plus")
  )
  return query_engine.query(query).response
# 测试一下效果
query_company_info("我们公司项目管理应该用什么工具")

'公司一般情况下应使用Microsoft Project作为项目管理工具，因为公司已购买完整的许可证。软件研发一组、三组和四组目前使用Jira，但计划在2026年之前逐步切换至Microsoft Project。'

尽管这些工具函数的实现比较简单，但却是大模型与外界交互的关键。接下来我们将使用Assistant API创建一个Agent，并将这三个工具函数集成进来。

2 将工具函数与大模型集成进Agent中

你已经定义好了工具函数，接下来就要将它们与大模型通过Assistant API集成到Agent中。

通过Assistants.create方法，你可以创建一个新的Agent，并通过model（模型名称）、name（Agent命名）、description（Agent的描述信息）、instructions（Agent非常重要的参数，用于提示Agent所具有的工具函数能力，同时也可以规范输出格式）、tools（工具函数通过该参数传入）参数来定义Agent。

其中，tools参数中的function.name用于指定工具函数，但需要为字符串格式，因此可以通过一个map方法映射到工具函数上。

In [33]:
# 引入依赖
from dashscope import Assistants, Messages, Runs, Threads
import json

# 定义贾维斯
ChatAssistant = Assistants.create(
  # 在此指定模型名称
  model="qwen-plus",
  # 在此指定Agent名称
  name="贾维斯",
  # 在此指定Agent的描述信息
  description="一个智能助手，能够查询员工信息，帮助员工发送请假申请，或者查询公司规章制度。",
  # 用于提示大模型所具有的工具函数能力，也可以规范输出格式
  instruction='''
  你是贾维斯，你的功能有以下三个：
    1. 查询员工信息。例如：查询员工张三的HR是谁；
    2. 发送请假申请。例如：当员工提出要请假时，你可以在系统里帮他完成请假申请的发送；
    3. 查询公司规章制度。例如：我们公司项目管理的工具是什么？
    请准确判断需要调用哪个工具，并礼貌回答用户的提问。
  ''',
  # 将工具函数传入
  tools =[
    {
      # 定义工具函数类型，一般设置为function即可
      'type': 'function',
      'function': {
        # 定义工具函数名称，通过map方法映射到query_employee_info函数
        'name': '查询员工信息',
        # 定义工具函数的描述信息，Agent主要根据description来判断是否需要调用该工具函数
        'description': '当需要查询员工信息时非常有用，比如查询员工张三的HR是谁，查询教育部门总人数等。',
        # 定义工具函数的参数
        'parameters': {
          'type': 'object',
          'properties': {
            # 将用户的提问作为输入参数
            'query': {
              'type': 'str',
              # 对输入参数的描述
              'description': '用户的提问'
            },
          },
          # 在此声明该工具函数需要哪些函数
          'required': ['query']
        },
      }
    },
    {
      'type': 'function',
      'function': {
        'name': '发送请假申请',
        'description': '当需要帮助员工发送请假申请时非常有用。',
        'parameters': {
          'type': 'object',
          'properties': {
            # 需要请假的时间
            'date': {
              'type': 'str',
              'description': '员工想要请假的时间'
            },
          },
          'required': ['date']
        },
      }
    },
    {
      'type': 'function',
      'function': {
        'name': '查询公司规章制度',
        'description': '当需要查询公司规章制度时非常有用，比如查询公司项目管理的工具是什么，查询公司都有哪些部门等。',
        'parameters': {
          'type': 'object',
          'properties': {
            'query': {
              'type': 'str',
              'description': '用户的提问'
            }
          }
        }
      }
    }
  ]
)
print(f'{ChatAssistant.name} 创建完成')

# 建立Agent Function name与工具函数的映射关系
function_mapper = {
  "查询员工信息": query_employee_info,
  "发送请假申请": send_leave_application,
  "查询公司规章制度": query_company_info
}

print('工具函数与function.name映射关系建立完成')

贾维斯 创建完成
工具函数与function.name映射关系建立完成


通过Assistant API获得Agent回复的过程需要涉及到如thread、message、run等概念，如果你对这些概念与细节感兴趣，请参考阿里云Assistant API官方文档。

我们封装好了一个辅助函数get_agent_response，你只需要传入Agent与提问内容，即可得到指定Agent的回复。

如果你希望给Agent配备更多的能力，可以添加工具函数，并在function_mapper与tools中建立映射关系。

In [36]:
# 输入message信息，输出为指定Agent的回复
def get_agent_response(assistant, message=''):
  # 打印出输入Agent的信息
  thread = Threads.create()
  message = Messages.create(thread.id, content=message)
  run = Runs.create(thread.id, assistant_id=assistant.id)
  run_status = Runs.wait(run.id, thread_id=thread.id)
  # 如果响应失败，会打印出run failed
  if run_status.status == 'failed':
    print('run failed:')
  # 如果需要工具来辅助大模型输出，则进行以下流程
  if run_status.required_action:
    f = run_status.required_action.submit_tool_outputs.tool_calls[0].function
    # 获得function name
    func_name = f['name']
    # 获得function 的入参
    param = json.loads(f['arguments'])
    # 打印出工具信息
    print("function is: ", f)
    # 根据function name，通过function_mapper映射到函数，并将参数输入工具函数得到output输出
    if func_name in function_mapper:
      output = function_mapper[func_name](**param)
    else:
      output = ""

    tool_outputs = [{
      'output': output
    }]
    run = Runs.submit_tool_outputs(run.id, thread_id=thread.id, tool_outputs=tool_outputs)
    run_status = Runs.wait(run.id, thread_id=thread.id)

  run_status = Runs.get(run.id, thread_id=thread.id)
  msgs = Messages.list(thread.id)
  # 将Agent的输出返回
  return msgs['data'][0]['content'][0]['text']['value']

我们可以向ChatAssistant提问，测试一下回复效果。

In [37]:
query_stk = [
  "谁是张三的HR？",
  "教育部门一共有多少员工？",
  "王五在哪个部门？",
  "帮我提交下周三请假的申请",
  "我们公司应该用什么项目管理工具呢？"
]

for query in query_stk:
  print("提问是: ")
  print(query)
  print("思考过程与最终输出是: ")
  print(get_agent_response(ChatAssistant, query))
  print("\n")

提问是: 
谁是张三的HR？
思考过程与最终输出是: 
function is:  {'name': '查询员工信息', 'arguments': '{"query": "谁是张三的HR？"}', 'output': None}
SQL statement is: select hr from employees where name = '张三';
很抱歉，目前我无法查询到张三的HR信息。建议您直接联系人力资源部门或通过公司内部系统进行查询。


提问是: 
教育部门一共有多少员工？
思考过程与最终输出是: 
function is:  {'name': '查询员工信息', 'arguments': '{"query": "教育部门一共有多少员工？"}', 'output': None}
SQL statement is: select count(*) from employees where department = '教育部门';
抱歉，目前我无法查询到教育部门的员工总数。请确认相关信息或稍后再试。


提问是: 
王五在哪个部门？
思考过程与最终输出是: 
function is:  {'name': '查询员工信息', 'arguments': '{"query": "王五在哪个部门"}', 'output': None}
SQL statement is: select department from employees where name = '王五'
王五在后勤部。


提问是: 
帮我提交下周三请假的申请
思考过程与最终输出是: 
function is:  {'name': '发送请假申请', 'arguments': '{"date": "下周三"}', 'output': None}
已为你发送请假申请，日期为下周三（2025年9月24日）。如有其他需要，请随时告知。


提问是: 
我们公司应该用什么项目管理工具呢？
思考过程与最终输出是: 
function is:  {'name': '查询公司规章制度', 'arguments': '{"query": "公司项目管理的工具"}', 'output': None}
根据公司的规定，项目管理工具有以下两种选择：

1. **Jira**：适用于软件开发团队，支持敏捷开发方法，如Scr

从测试结果可以看出，拓展了功能之后的答疑机器人达到了你预期的效果。